In [1]:
#!/usr/bin/env python

import numpy as np
import cv2
from numpy.core.numeric import full
import rospy
import rospkg
import open3d as o3d
from transforms3d.quaternions import mat2quat
import time
import copy
import os
from copy import deepcopy

import sys
from os import path
from ocrtoc_common.camera_interface import CameraInterface
from ocrtoc_common.transform_interface import TransformInterface
from sensor_msgs.msg import CameraInfo, Image, PointCloud2
import pickle

from src.ocrtoc_perception.graspnet import GraspNetBaseLine
import open3d_plus as o3dp

WARNING - 2021-12-10 18:30:54,082 - rigid_transformations - autolab_core not installed as catkin package, RigidTransform ros methods will be unavailable
INFO - 2021-12-10 18:30:54,264 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


## Load point cloud

In [90]:
if __name__ == "__main__" :
    rospack = rospkg.RosPack()
    taskid= '1-1-1'
    save_path = os.path.join(rospack.get_path('ocrtoc_perception'),'data',taskid, '')
    full_pcd = o3d.io.read_point_cloud(save_path + 'full_pcd.pcd') 
    with open(save_path +  'color_images.pickle', 'rb') as handle:
        color_images = pickle.load(handle)  
    with open(save_path + 'camera_poses.pickle', 'rb') as handle:
        camera_poses = pickle.load(handle) 
    # Compute Grasping Poses (Many Poses in a Scene)
    
    with open(save_path + 'gg.pickle', 'rb') as handle:
        gg = pickle.load(handle)
     
        
        
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1) 

## load and run Graspnet

In [105]:
# this was found through perceptionservice enterence -> rosparam -> file pointer
graspnet_chekpoint_path = "src/ocrtoc_perception/graspnet/logs/log_rs/checkpoint.tar"
redefine_approach = 0
"""
graspnet_baseline = GraspNetBaseLine(
            checkpoint_path = os.path.join(
                rospkg.RosPack().get_path('ocrtoc_perception'),
                graspnet_chekpoint_path),collision_thresh=0.001,voxel_size=0.01)
"""
"""
Modified
"""
graspnet_baseline = GraspNetBaseLine(
            checkpoint_path = os.path.join(
                rospkg.RosPack().get_path('ocrtoc_perception'),
                graspnet_chekpoint_path),collision_thresh=0.001,empty_thresh=0,voxel_size=1)


def compute_grasp_pose(full_pcd):
        points, _ = o3dp.pcd2array(full_pcd)
        grasp_pcd = copy.deepcopy(full_pcd)
        grasp_pcd.points = o3d.utility.Vector3dVector(-points)

        # generating grasp poses.
        gg = graspnet_baseline.inference(grasp_pcd)
        gg.translations = -gg.translations
        gg.rotation_matrices = -gg.rotation_matrices
        gg.translations = gg.translations + gg.rotation_matrices[:, :, 0] * redefine_approach
        gg = graspnet_baseline.collision_detection(gg, points)

        # all the returned result in 'world' frame. 'gg' using 'graspnet' gripper frame.
        return gg


gg = compute_grasp_pose(full_pcd)
print(f"Managed to generate {len(gg)} grasps")

-> loaded checkpoint /root/ocrtoc_ws/src/ocrtoc_perception/src/ocrtoc_perception/graspnet/logs/log_rs/checkpoint.tar (epoch: 18)
Managed to generate 54 grasps


## Visualize

In [106]:
o3d.visualization.draw_geometries([frame, full_pcd,*gg.to_open3d_geometry_list()])